In [27]:
import pandas as pd
import numpy as np
import copy
import requests
import json
from datetime import timedelta, date, datetime
import os

## Read in and clean the latest COVID data

In [28]:
data = requests.get('http://pomber.github.io/covid19/timeseries.json').json()

In [29]:
#Generate dataframe from json data: Cumulative data
keys_countries= data.keys() 
df=pd.DataFrame()
for key in data:
    df1= pd.json_normalize(data[key])
    df1['Country']=key
    df= df.append(df1)



In [30]:
df

,date,confirmed,deaths,recovered,Country
0,2020-1-22,0,0,0,Afghanistan
1,2020-1-23,0,0,0,Afghanistan
2,2020-1-24,0,0,0,Afghanistan
3,2020-1-25,0,0,0,Afghanistan
4,2020-1-26,0,0,0,Afghanistan
...,...,...,...,...,...
541,2021-7-16,81168,2520,51139,Zimbabwe
542,2021-7-17,82613,2588,51923,Zimbabwe
543,2021-7-18,83619,2622,53453,Zimbabwe
544,2021-7-19,85732,2697,55714,Zimbabwe


In [31]:
# Dataframe with daily cases per country (above is cumulatve)
keys_countries= data.keys() 
df_daily=pd.DataFrame()
for key in data:
    df1= pd.json_normalize(data[key])
    df1.set_index(['date'],inplace=True)
    df2=copy.deepcopy(df1)
    for col in df2.columns:
        df2[col]=df2[col]- df2[col].shift(1)
    df2['Country']=key
    df_daily= df_daily.append(df2)
df_daily.reset_index(inplace=True)


## Adjust cumulative and daily COVID data per hundred thousand population

In [32]:
# Get population data
url = 'https://data.un.org/_Docs/SYB/CSV/SYB63_1_202105_Population,%20Surface%20Area%20and%20Density.csv'
df_data= pd.read_csv(url, header=1)


In [33]:
#df with just country and population, convert from ('millions')
df_pops=df_data[(df_data['Year']==2019)&(df_data['Series']==df_data.Series[0])][['Unnamed: 1','Value']]
df_pops.columns=['Country', 'Population']
df_pops.Population = df_pops.Population*1000000

In [34]:
#Convert country names if different in population vs covid datasets

#Create dictionary for names to change 
countries_pops=pd.unique(df_pops.Country)

countries_namechange= []
for country in (pd.unique(df.Country)):
    if country not in (pd.unique(df_pops.Country)):
        countries_namechange.append(country)
        
countries_namechange
new_names={}
for country in countries_namechange:
    to_check= country[:4]
    result = [v for v in countries_pops if to_check in v]
    if result:
        print(result)
        print(country + ': type index to keep or else F')
        x = input()
        if x == 'F':
            new=country
            print('no match')
        else:
            new=result[int(x)]
            print('{} is changed to {}'.format(country, new))
    else:
        print(country, ': no match')
        new=country
    new_names[country]= new



['Bolivia (Plurin. State of)', 'Venezuela (Boliv. Rep. of)']
Bolivia: type index to keep or else F


KeyboardInterrupt: Interrupted by user

In [ ]:
#save to json file
with open('../data/interim/country_namechanges.json', 'w') as fp:
    json.dump(new_names, fp)

In [35]:
#after doing this once, just load the file
with open('../data/interim/country_namechanges.json', 'r') as fp:
    new_names = json.load(fp)

In [36]:
#convert the names in covid data to names in the population data
new_names['US']='United States'
new_names['United States of America']='United States'
for key in new_names:
    df.replace(to_replace=key, value=new_names[key], inplace=True)
    df_daily.replace(to_replace=key, value=new_names[key], inplace=True)
    df_pops.replace(to_replace=key, value=new_names[key], inplace=True)
    df_data.replace(to_replace=key, value=new_names[key], inplace=True)

In [37]:
#convert dates to datetime objects
df_daily.date = pd.to_datetime(df_daily.date)
df.date = pd.to_datetime(df.date)


In [38]:
#Add populations to the cumulative and daily COVID dataframes
cumulative_df_pops = pd.merge(df, df_pops, on='Country')
daily_df_pops = pd.merge(df_daily, df_pops, on='Country')

In [39]:
#consistent column order for daily and cumulative dfs
col_order=cumulative_df_pops.columns
daily_df_pops = daily_df_pops[col_order]


In [40]:
# Create dataframes with cases and deaths per hundred thousand of each country's population (cumulative and daily values)
per100Th_cumulative = copy.deepcopy(cumulative_df_pops)

for col in col_order[1:4]:
    per100Th_cumulative[col]= per100Th_cumulative[col]*100000/(per100Th_cumulative['Population']).apply(pd.to_numeric, errors='coerce')
per100Th_daily=copy.deepcopy(daily_df_pops)
for col in col_order[1:4]:
    per100Th_daily[col]= per100Th_daily[col]*100000/(per100Th_daily['Population']).apply(pd.to_numeric, errors='coerce')


In [41]:
df.to_csv('../data/interim/cumulative_coviddata.csv')
df_daily.to_csv('../data/interim/daily_coviddata.csv')
per100Th_cumulative.to_csv('../data/interim/cumulative_PHT_coviddata.csv')
per100Th_daily.to_csv('../data/interim/daily_PHT_coviddata.csv')

In [48]:
df.iloc[0,0]

Timestamp('2020-01-22 00:00:00')